In [1]:
import os
import sys
import time
import requests
from py2neo import Graph, Node, Relationship
from bs4 import BeautifulSoup
import urllib.request
import csv 
from neo4j.v1 import GraphDatabase, basic_auth, TRUST_ON_FIRST_USE, CypherError 
import networkx as nx
import cypher


In [2]:
graph = Graph(password="a1b2c3")

In [3]:
urlpage =  'http://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'

In [4]:
page = urllib.request.urlopen(urlpage)

In [5]:
soup = BeautifulSoup(page, 'html.parser')

In [6]:
print(soup)

<!-- Template Name: League Table page
-->
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if gt IE 9]><!--><html class="no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"><!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link data-minify="1" href="https://www.fasttrack.co.uk/wp-content/cache/min/1/18d526ce995aa32bdf2b3acb067f5a3b.css" rel="stylesheet"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet) & Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>
		League table - Fast Track	</t

In [7]:
# find results within table
table = soup.find('table', attrs={'class': 'tableSorter'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 101


In [8]:
# create and write headers to a list 
rows = []
rows.append(['Rank', 'Company_Name', 'Webpage', 'Description', 'Location', 'Year_end', 'Annual_sales_rise_over_3_years', 'Sales', 'Staff', 'Comments'])
print(rows)

[['Rank', 'Company_Name', 'Webpage', 'Description', 'Location', 'Year_end', 'Annual_sales_rise_over_3_years', 'Sales', 'Staff', 'Comments']]


In [9]:
# loop over results
for result in results:
    
    data = result.find_all('td')
     
    if len(data) == 0: 
        continue
    
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearend = data[3].getText()
    salesrise = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()
    print('Company is', company)
    print('Sales', sales)
    
    companyname = data[1].find('span', attrs={'class':'company-name'}).getText()    
    description = company.replace(companyname, '')
    sales = sales.strip('*').strip('†').replace(',','')
    url = data[1].find('a').get('href')
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    try:
        tableRow = soup.find('table').find_all('tr')[-1]
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None
        
    rows.append([rank, companyname, webpage, description, location, yearend, salesrise, sales, staff, comments])


Company is Plan.comCommunications provider
Sales *35,418
Company is PsiOxusBiotechnology developer
Sales 53,136
Company is CensorNetCloud security software developer
Sales *7,535
Company is thoughtonomyAutomation software developer
Sales *16,916
Company is PerkboxEmployee engagement services
Sales *34,700
Company is OguryMobile data marketing
Sales 33,760
Company is VerveMarketing software developer
Sales *10,747
Company is goHenryChildren's pre-paid card developer
Sales *6,116
Company is DarktraceCybersecurity developer
Sales *58,573
Company is BizumaB2B e-commerce platform
Sales *9,608
Company is DepopSocial shopping app
Sales *94,300
Company is Laser Wire SolutionsLaser wire-stripping machinery
Sales *5,114
Company is Bought By ManySpecialist insurance platform
Sales *19,727
Company is OptalPayment services provider
Sales 177,653
Company is InfinoxCurrency trading platform
Sales *41,700
Company is OakbrookConsumer finance provider
Sales *40,400
Company is CarwowCar sales platform
Sa

In [10]:
print(rows)

[['Rank', 'Company_Name', 'Webpage', 'Description', 'Location', 'Year_end', 'Annual_sales_rise_over_3_years', 'Sales', 'Staff', 'Comments'], ['1', 'Plan.com', 'http://www.plan.com', 'Communications provider', 'Isle of Man', 'Sep-17', '364.38%', '35418', '90', 'About 650 partners use its telecoms platform to support more than 100,000 UK business customers'], ['2', 'PsiOxus', 'http://www.psioxus.com', 'Biotechnology developer', 'Oxfordshire', 'Dec-17', '311.67%', '53136', '54', 'Received a $15m milestone payment from its development partner Bristol-Myers Squibb in December'], ['3', 'CensorNet', 'http://www.censornet.com', 'Cloud security software developer', 'Basingstoke', 'Dec-17', '210.17%', '7535', '77', 'Has more than 4,000 customers, including McDonald’s in France and the European Central Bank'], ['4', 'thoughtonomy', 'http://www.thoughtonomy.com', 'Automation software developer', 'East London', 'May-18', '205.20%', '16916', '100', 'It sells to 28 countries and 50% of revenue is gen

In [11]:
# Create csv and write rows to output file
with open('techtrack100.csv','w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)

In [ ]:
import shutil
neo4j_import_folder = ''
# example- neo4j_import_folder = 'F:/New folder'
shutil.copy('techtrack100.csv', neo4j_import_folder)

In [18]:
query = '''
LOAD CSV WITH HEADERS FROM "file:///techtrack100.csv" AS row
CREATE (:Customer {Rank: row.Rank, Company_Name: row.Company_Name, Webpage: row.Webpage, Description: row.Description, Location: row.Location, Year_end: row.Year_end, Annual_sales_rise_over_3_years: row.Annual_sales_rise_over_3_years, Sales: row.Sales, Staff: row.Staff, Comments: row.Comments});
'''

In [19]:
graph.run(query)